In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
train_path = 'C://Users/Soumyajit/Downloads/datasets/Colorization of Image Data'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1/255, rotation_range = 50, horizontal_flip= True, vertical_flip = True)

In [4]:
train_data = train_datagen.flow_from_directory(train_path, target_size= (256,256), batch_size = 256, shuffle = True)

Found 63566 images belonging to 2 classes.


In [5]:
train_image, label = train_data.next()

In [6]:
X =[]
Y =[]

for img in train_image:
    try:
        lab = rgb2lab(img)
        X.append(lab[:,:,0])
        Y.append(lab[:,:,1:] / 128)
    except:
        print('error')

In [7]:
X_train = np.array(X)
X_train = X_train.reshape(X_train.shape+(1,))

y_train = np.array(Y)

In [8]:
print(X_train.shape)
print(y_train.shape)

(256, 256, 256, 1)
(256, 256, 256, 2)


In [ ]:
vggmodel = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape=(256, 256, 3))

In [10]:
for layer in vggmodel.layers[:19]:
    layer.trainable = False

In [11]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [12]:
vggfeatures = []
for i, sample in enumerate(X_train):
    sample = gray2rgb(sample)
    sample = sample.reshape((1,256,256,3))
    prediction = vggmodel.predict(sample)
    prediction = prediction.reshape((8,8,512))
    vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)

In [13]:
vggfeatures.shape

(256, 8, 8, 512)

In [14]:
encoder_input = Input(shape=(8, 8, 512,))
decoder_output = Conv2DTranspose(256, (3,3), activation='relu', padding='same')(encoder_input)
decoder_output = Conv2DTranspose(128, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2DTranspose(2, (3, 3), activation='tanh', padding='same')(decoder_output)
output = UpSampling2D((2, 2))(decoder_output)

In [15]:
model = Model(inputs=encoder_input, outputs=output)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 512)]       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 256)         1179904   
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 128)         295040    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 16, 16, 64)        73792     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 32, 32, 32)        18464 

In [17]:
model.compile(optimizer =Adam(learning_rate = 0.0002), loss='mse' , metrics=['accuracy'])

In [18]:
model.fit(vggfeatures, y_train, batch_size=8, epochs=30, verbose= 2)

Epoch 1/30
32/32 - 22s - loss: 0.0623 - accuracy: 0.5473
Epoch 2/30
32/32 - 0s - loss: 0.0089 - accuracy: 0.6626
Epoch 3/30
32/32 - 0s - loss: 0.0080 - accuracy: 0.6732
Epoch 4/30
32/32 - 0s - loss: 0.0077 - accuracy: 0.6893
Epoch 5/30
32/32 - 0s - loss: 0.0076 - accuracy: 0.6930
Epoch 6/30
32/32 - 0s - loss: 0.0075 - accuracy: 0.7336
Epoch 7/30
32/32 - 0s - loss: 0.0074 - accuracy: 0.7321
Epoch 8/30
32/32 - 0s - loss: 0.0074 - accuracy: 0.7481
Epoch 9/30
32/32 - 0s - loss: 0.0073 - accuracy: 0.7561
Epoch 10/30
32/32 - 0s - loss: 0.0073 - accuracy: 0.7595
Epoch 11/30
32/32 - 0s - loss: 0.0073 - accuracy: 0.7654
Epoch 12/30
32/32 - 0s - loss: 0.0072 - accuracy: 0.7654
Epoch 13/30
32/32 - 0s - loss: 0.0072 - accuracy: 0.7715
Epoch 14/30
32/32 - 0s - loss: 0.0071 - accuracy: 0.7717
Epoch 15/30
32/32 - 0s - loss: 0.0070 - accuracy: 0.7685
Epoch 16/30
32/32 - 0s - loss: 0.0068 - accuracy: 0.7739
Epoch 17/30
32/32 - 0s - loss: 0.0067 - accuracy: 0.7650
Epoch 18/30
32/32 - 0s - loss: 0.0065 -

In [ ]:
img1_color = []

path = "C://Users/Soumyajit/Downloads/Auto Encoder/test11.jpg"

img = img_to_array(load_img(path))
img = resize(img, (256,256), anti_aliasing=True)
img*= 1.0/255

plt.title("Input Image")
imshow(img)
plt.axis('off')
plt.show()

In [ ]:
lab = rgb2lab(img)
l = lab[:,:,0]
L = gray2rgb(l)
L = L.reshape((1,256,256,3))

In [ ]:
output = vggmodel.predict(L)
ab = model.predict(output)
ab = ab * 128

In [ ]:
cur = np.zeros((256, 256, 3))
cur[:,:,0] = l
cur[:,:,1:] = ab

In [ ]:
imshow(lab2rgb(cur))